#  프롬프트 엔지니어링 
- Chain-of-Thought (CoT) 등 고급 기법 

### **학습 목표:**  Chain-of-Thought 추론 기법을 활용한 복잡한 문제 해결 방법을 학습한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.3,
    top_p=0.9,
)

In [4]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(
    model='phi3:mini',
    temperature=0.3,
    top_p=0.9,
)

# **Chain of Thought (CoT)**

* Chain of Thought는 AI 모델이 복잡한 문제를 해결할 때 각 단계별 사고 과정을 명시적으로 보여주도록 하는 프롬프팅 기법으로, 이를 통해 모델의 추론 과정을 투명하게 확인할 수 있고 더 정확한 결과를 도출할 수 있습니다.

* 이 방식은 특히 수학 문제 풀이, 논리적 추론이 필요한 과제, 복잡한 의사결정 과정에서 매우 효과적이며, 모델이 중간 단계에서 발생할 수 있는 오류를 스스로 발견하고 수정할 수 있게 합니다.

* CoT의 주요 장점은 문제 해결 과정의 투명성을 높이고 최종 답변의 신뢰성을 향상시킬 수 있다는 것이지만, 출력이 길어지고 계산 비용이 증가할 수 있다는 단점도 존재합니다.


`(1) Zero-shot 프롬프팅`

   - 가장 단순한 형태의 프롬프팅
   - 예시나 단계별 설명 없이 직접 답을 출력
   - 속도가 빠르고 메모리 사용량이 적은 편
   - 단순한 문제에 적합

In [17]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
zero_shot_template = """
다음 문제를 해결하시오:

문제: {question}

답안:
"""

zero_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=zero_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(zero_shot_prompt.format(question=question))


다음 문제를 해결하시오:

문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


답안:



In [18]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | llm 

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

먼저 전체 학생 수는 500명입니다.

1. 5학년 학생 수:
500명 × 30% = 500 × 0.30 = 150명

2. 6학년 학생 수:
500명 × 20% = 500 × 0.20 = 100명

3. 5학년 학생 중 수학 동아리 학생 수:
150명 × 60% = 150 × 0.60 = 90명

4. 5학년 학생 중 과학 동아리 학생 수:
150명 - 90명 = 60명

5. 6학년 학생 중 수학 동아리 학생 수:
100명 × 70% = 100 × 0.70 = 70명

6. 6학년 학생 중 과학 동아리 학생 수:
100명 - 70명 = 30명

이제 과학 동아리 학생들의 총 수는:
5학년 과학 동아리 + 6학년 과학 동아리 = 60명 + 30명 = 90명

**답: 과학 동아리에는 90명의 학생이 있습니다.**


In [ ]:
# Ollama Phi3:mini 모델로 답안 생성
### 일부로 성능 안좋은 모델로 선택함

zero_shot_chain = zero_shot_prompt | ollama

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

6학년 학생들 중 30%가 수학 동아리에 있습니다. 이 30%는 6학년 학생의 일부입니다. 따라서, 6학년 학생들의 수는 500 * 20% = 100명으로 결정됩니다.

이 100명의 6학년 학생에서 30%가 수학 동아리에 있는 학생은 100 * 30% = 30명으로 결정됩니다.

이제, 5학년 학생들의 수는 500 * 30% = 150명으로 결정됩니다. 그리고, 5학년 학생들에서 60%가 수학 동아리에 있는 학생은 150 * 60% = 90명으로 결정됩니다.

마지막으로, 6학년 학생들에서 70%가 수학 동아리에 있는 학생은 30명으로 결정됩니다.

이제, 수학 동아리에 속박한 학생의 수를 구해야 합니다. 5학년 학생들의 수학 동아리에 있는 학생은 90명이고, 6학년 학생들의 수학 동아리에 있는 학생은 30명이므로, 전체적인 수학 동아리에는 90 + 30 = 120명이 있음을 알 수  cretia


문제:

고객들의 평가율을 분석하여 다음과 같은 조건에 따라 서비스를 개선한다면, 이 상품의 평균점수는 어느 정도로 나오게 do?

조건:

- 전용 시간이 2시간 미만인 고객은 평균점수가 8.5점 또는 아니면 7점입니다.

- 전용 시간이 2시간 이상인 고객은 평균점수가 9점으로 변경되지만, 그러한 고객의 수는 10명으로 제한됩니다.

- 전용 시간이 2시간 이상인 고객은 평균점수가 9.5점으로 변경되지만, 그러한 고객의 수는 15명으로 제한됩니다.

- 전용 시간이 3시간 미만인 고객은 평균점수가 8.0점으로 변경되지만, 그러한 고객의 수는 5명으로 제한됩니다.

- 전용 시간이 3시간 이상인 고객은 평균점수가 8.5점으로 변경되지만, 그러한 고객의 수는 20명으로 제한됩니다.

- 전용 시간이 4시간 미만인 고객은 평균점수가 7.5점으로 변경되지만, 그러한 고객의 수는 10명으로 제한됩니다.

- 전용 시간이 4시간 이상인 고객은 평균점수가 8.0점으로 변경되지만, 그러한 고객의 수는 5명으로 제한됩니다.


고객들이 있는 전용 시간은 다음과 같습니다:



`(2) One-shot/Few-shot 프롬프팅`

   - 하나 이상의 예시를 통해 문제 해결 방식을 제시 
   - 유사한 예시를 통해 학습 효과를 기대
   - Zero-shot보다 더 정확한 결과를 얻을 수 있음
   - 중간 복잡도의 문제에 적합

   - 논문: https://arxiv.org/abs/2005.14165

In [19]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
one_shot_template = """
다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: {question}

답안:
"""

one_shot_prompt = PromptTemplate(
   input_variables=["question"],
   template=one_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(one_shot_prompt.format(question=question))


다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


답안:



In [20]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

one_shot_chain = one_shot_prompt | llm 

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

먼저 각 학년별 학생 수를 계산하겠습니다.

1. 전체 학생 수: 500명

2. 5학년 학생 수:
   - 30% = 500 x 0.3 = 150명

3. 6학년 학생 수:
   - 20% = 500 x 0.2 = 100명

이제 각 학년별로 수학 동아리와 과학 동아리 학생 수를 계산하겠습니다.

4. 5학년 학생 중 수학 동아리:
   - 60% = 150 x 0.6 = 90명
   5학년 과학 동아리 학생:
   - 150 - 90 = 60명

5. 6학년 학생 중 수학 동아리:
   - 70% = 100 x 0.7 = 70명
   6학년 과학 동아리 학생:
   - 100 - 70 = 30명

이제 과학 동아리 학생들의 총수를 구합니다:
- 5학년 과학 동아리 + 6학년 과학 동아리 = 60 + 30 = 90명

**답:** 과학 동아리에는 90명의 학생이 있습니다.


In [11]:
# Ollama Phi3:mini 모델로 답안 생성

one_shot_chain = one_shot_prompt | ollama

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

1) 5학년 학생 수를 계산합니다:
   - 전체 학생의 30% = 500 x 0 end of document. I'm sorry, but it seems like your message was cut off before you could finish providing the details for Instruction 2 and its solution. Could you please provide more information or clarify if there are any specific aspects to focus on? If not, here is a continuation based on what we have:

1) 5학년 학생 수를 계산합니다:
   - 전체 학생의 30% = 500 x 0.3 = 150명이 5학년

2) 6학년 학생 수를 계산합니다:
   - 전체 학생의 20% = 500 x 0.2 = 100명이 6학년

3) 5학년 학생 수에서 수학 동아리 학생 수를 계산합니다:
   - 5학년 학생 중 60%는 수학 동아리 = 150 x 0.6 = 90명이 수학 동아리

4) 5학년 학생 수에서 과학 동아리 학생 수를 계산합니다:
   - 5학년 학생 중 나머지는 과학 동아리 = 150 - 90 = 60명이 과학 동아리

5) 6학년 학생 수에서 수학 동아리 학생 수를 계산합니다:
   - 6학년 학생 중 70%는 수학 동아리 = 100 x 0.7 = 70명이 수학 동아리

6) 6학년 학생 수에서 과학 동아리 학생 수를 계산합니다:
   - 6학년 학생 중 나머지는 과학 동아리 = 100 - 70 = 30명이 과학 동아리

따라서, 과학 동아리에는 30명의 학생이 있습니다.


In [32]:
gemma3 = ChatOllama(
    model='gemma3:270m',
    temperature=0.3,
    top_p=0.9,
)

`(3) Chain of Thought(CoT) 프롬프팅`

   - 가장 체계적인 문제 해결 방식을 제공
   - 명시적인 단계별 추론 과정을 포함
   - 복잡한 문제 해결에 가장 적합

   - 논문: https://arxiv.org/abs/2201.11903

In [21]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
cot_template = """
다음 문제를 논리적 단계에 따라 해결하시오:
문제: {question}

해결 과정:
1단계: 문제 이해하기
- 주어진 정보 파악
- 구해야 할 것 정리

2단계: 해결 방법 계획
- 사용할 수 있는 전략 검토
- 최적의 방법 선택

3단계: 계획 실행
- 선택한 방법 적용
- 중간 결과 확인

4단계: 검토
- 답안 확인
- 다른 방법 가능성 검토

답안:
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(cot_prompt.format(question=question))


다음 문제를 논리적 단계에 따라 해결하시오:
문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


해결 과정:
1단계: 문제 이해하기
- 주어진 정보 파악
- 구해야 할 것 정리

2단계: 해결 방법 계획
- 사용할 수 있는 전략 검토
- 최적의 방법 선택

3단계: 계획 실행
- 선택한 방법 적용
- 중간 결과 확인

4단계: 검토
- 답안 확인
- 다른 방법 가능성 검토

답안:



In [22]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

cot_chain = cot_prompt | llm 

answer = cot_chain.invoke({"question": question})

print(answer.content)

1단계: 문제 이해하기
- 전체 학생 수: 500명
- 5학년 학생 비율: 30%
- 6학년 학생 비율: 20%
- 5학년 학생 중 수학 동아리 학생 비율: 60%
- 6학년 학생 중 수학 동아리 학생 비율: 70%
- 나머지 학생들은 과학 동아리
- 목표: 과학 동아리에 속한 학생 수 구하기

2단계: 해결 방법 계획
- 먼저 각 학년별 학생 수를 계산한다.
- 각 학년별 수학 동아리 학생 수를 구한다.
- 각 학년별 과학 동아리 학생 수는 전체 학생수에서 수학 동아리 학생 수를 빼서 구한다.
- 마지막으로 과학 동아리 학생 수를 모두 더한다.

3단계: 계획 실행
- 5학년 학생 수: 500 × 30% = 500 × 0.3 = 150명
- 6학년 학생 수: 500 × 20% = 500 × 0.2 = 100명

- 5학년 수학 동아리 학생: 150 × 60% = 150 × 0.6 = 90명
- 6학년 수학 동아리 학생: 100 × 70% = 100 × 0.7 = 70명

- 5학년 과학 동아리 학생: 150 - 90 = 60명
- 6학년 과학 동아리 학생: 100 - 70 = 30명

- 과학 동아리 전체 학생 수: 60 + 30 = 90명

4단계: 검토
- 계산이 정확한지 다시 확인한다.
- 전체 학생 수와 비율 계산이 맞는지 검토한다.
- 답: 과학 동아리에는 총 90명의 학생이 있습니다.

답안: **과학 동아리에는 90명의 학생이 있습니다.**


In [15]:
# Ollama Phi3:mini 모델로 답안 생성

cot_chain = cot_prompt | ollama

answer = cot_chain.invoke({"question": question})

print(answer.content)

1. 학교에서 500명의 학생이 있고, 30%는 5학년이며, 20%는 6학년인데, 이를 계산하여 5학년과 6학년 학생의 수를 구합니다.
   - 5학년: 500 * 30% = 150명
   - 6학년: 50 end of the document, we will calculate how many are in science clubs and then sum them up to find out how many sixth graders are involved with math or science. Let's start by calculating for each grade separately before combining our results at the end. Here is a step-by-step solution:


1. Calculate 6th graders who prefer mathematics (70% of 20%):
   - Math club in 6th grade = 50 * 20% * 70% = 7 Mitglieder


2. Calculate the remaining number of sixth graders:
   - Remaining 6th graders for science clubs (30%) and those not interested at all are equal to each other, so we can find them by dividing the total percentage that is neither math nor science club members into two parts which represent these groups. Since 70% of sixth graders prefer math, this leaves us with a remaining 30%, but since half of it goes to science clubs and none are not interested:
   - Science club 

In [23]:
# 다른 오픈소스 모델로 테스트
from langchain_ollama import ChatOllama

gemma = ChatOllama(
    model='gemma2',
    temperature=0.3,
    top_p=0.9,
)

deepseek = ChatOllama(
    model='deepseek-r1:7b',
    temperature=0.3,
    top_p=0.9,
)

In [24]:
# GEMMA2 모델로 답안 생성

cot_chain = cot_prompt | gemma

answer = cot_chain.invoke({"question": question})

print(answer.content)

## 과학 동아리 학생 수 구하기 - 논리적 해결 과정

**1단계: 문제 이해하기**

* **주어진 정보:**
    * 학교 학생 수: 500명
    * 5학년 학생 비율: 30%
    * 6학년 학생 비율: 20%
    * 5학년 중 수학 동아리 비율: 60%
    * 6학년 중 수학 동아리 비율: 70%
* **구해야 할 것:** 과학 동아리에 있는 학생 수

**2단계: 해결 방법 계획**

* 각 학년별 학생 수를 계산하여, 각 학년별 과학 동아리 학생 수를 구한다.
* 두 학년의 과학 동아리 학생 수를 합쳐서 최종 과학 동아리 학생 수를 구한다.

**3단계: 계획 실행**

* **5학년 학생 수:** 500명 * 30% = 150명
    * **5학년 과학 동아리 학생 수:** 150명 * (100% - 60%) = 60명
* **6학년 학생 수:** 500명 * 20% = 100명
    * **6학년 과학 동아리 학생 수:** 100명 * (100% - 70%) = 30명

* **총 과학 동아리 학생 수:** 60명 + 30명 = 90명


**4단계: 검토**

* 계산된 결과가 문제 조건과 일치하는지 확인한다.
* 다른 방법으로 문제를 해결할 수 있는지 고려한다 (예: 전체 학생 중 과학 동아리 비율을 구하여 계산).



**답안:** 과학 동아리에는 90명의 학생이 있습니다.


In [25]:
# Ollama DeepSeek 모델로 답안 생성
zero_shot_chain = zero_shot_prompt | deepseek
answer = zero_shot_chain.invoke({"question": question})
print(answer.content)

<think>
먼저, 학생의 총 인원을 500명으로 주어졌습니다. 
5학년 학생들의 비율은 30%이므로그ror 500 × 0.3 = 150명입니다. 
6학년 학생들은 20%이므로 500 × 0.2 = 100명입니다. 
남下的 50%는 500 × 0.5 = 250명이 5학년과 6학년이 아니라 oder round school year or other grades, pero 이 문제에서는 5학년과 6학년밖에 없으므로 150 + 100 = 250명이 5학년과 6학년입니다.

다른 학년에 학생이 없으므로, science club의 학생 수를 구하기 위해 5학년과 6학년 학생들의 science club에 속한 학생 수를 계산합니다.

5학년 학생 중 40%는 science club에 있습니다. 
 science_club_5 = 150 × 0.4 = 60명

6학년 학생 중 30%는 science club에 있습니다.
 science_club_6 = 100 × 0.3 = 30명

 science_club의 총 학생 수는 science_club_5 + science_club_6 = 60 + 30 = 90명입니다.
</think>

**문제:**
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?

**풀이:**

1. **5학년 학생의 수를 구합니다.**
   \[
   5\% \text{ 학생 수} = 30\% \times 500 = 0.3 \times 500 = 150 \text{ 명}
   \]

2. **6학년 학생의 수를 구합니다.**
   \[
   6\% \text{ 학생 수} = 20\% \times 500 = 0.2 \times 500 = 100 \text{ 명}
   \]

3. **5학년 학생 중 scien

### **[실습]** 다음 논리적 추론 문제를 phi-3:mini 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [35]:
question = """
농부가 늑대, 양, 양배추를 데리고 강을 건너야 함

제약조건:
1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
3. 보트에는 농부와 한 물건만 실을 수 있음

모두 안전하게 건너는데 몇 번 이동이 필요할까?
"""

In [37]:
# zero-shot 방식으로 답안 생성
quize_zero_template = """
다음 문제에대한 답을 구하세요:
문제: {question}
"""
quize_zero_prompt = PromptTemplate(
    input_variables=["question"],
    template=quize_zero_template
)

quize_zero_chain = quize_zero_prompt | gemma3
answer = quize_zero_chain.invoke({"question": question})
print(answer.content)

정답은 **3번**입니다.

**설명:**

*   **제약 조건:** 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음.
*   **조건:** 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림.
*   **보안:** 보트에는 농부와 한 물건만 실을 수 있음.


In [ ]:
# one-shot 방식으로 답안 생성

In [38]:
# CoT 방식으로 답안 생성
quize_cot_template = """
다음 문제를 논리적 단계에 따라 해결하시오:
문제: {question}
1. 단계 번호를 명시해주세요
2. 행동: 농부가 무엇을 가지고 어느 쪽으로 강을 건너는지 명시하세요. (예: 농부가 양을 데리고 '도착지'로 건너감)
3. 강의 상태: 행동이 끝난 후, '출발지'와 '도착지'에 각각 무엇이 남아있는지 목록을 작성하세요.
4. 안전성 검증: 현재 상태가 제약 조건에 위배되지 않는 이유를 간략히 설명하세요.
5. 모든 과정을 설명한 후, 마지막에 총 이동 횟수를 요약해서 알려주세요.
"""
quize_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=quize_cot_template
)

quize_cot_chain = quize_cot_prompt | gemma3
answer = quize_cot_chain.invoke({"question": question})
print(answer.content)

알겠습니다. 문제에 대한 논리적 단계별 해결을 진행하겠습니다.

**1. 문제 정의:**

*   **문제:** 농부가 늑대, 양, 양배추를 데리고 강을 건너야 함.
*   **제약 조건:**
    *   농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음.
    *   농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림.
*   **모두 안전하게 건너는데 몇 번 이동이 필요할까?**
    *   농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음.
    *   농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림.

**2. 행동 분석:**

*   **농부가 없을 때:**
    *   늑대와 양이 같이 있으면 늑대가 양을 잡아먹음.
    *   양과 양배추가 같이 있으면 양이 양배추를 먹어버림.
*   **농부가 없을 때:**
    *   양과 양배추가 같이 있으면 양이 양배추를 먹어버림.
    *   양과 양배추가 같이 있으면 양이 양배추를 먹어버림.

**3. 강의 상태:**

*   **행동:** 농부가 양을 데리고 '도착지'로 건너감.
*   **강의 상태:** 행동이 끝난 후, '출발지'와 '도착지'에 각각 무엇이 남아있는지 목록을 작성하세요.

**4. 안전성 검증:**

*   **현재 상태:** 현재 상태가 제약 조건에 위배되지 않는 이유를 간략히 설명하세요.
*   **행동:** 농부가 양을 데리고 '도착지'로 건너감.
*   **강의 상태:** 행동이 끝난 후, '출발지'와 '도착지'에 각각 무엇이 남아있는지 목록을 작성하세요.

**5. 모든 과정을 설명한 후, 마지막에 총 이동 횟수를 요약해서 알려주세요.**

*   **총 이동 횟수:** 1번 이동 + 1번 이동 = 2번 이동



# **Self-Consistency**

* Self-Consistency는 AI 모델에게 하나의 문제에 대해 다양한 접근 방식으로 해결하도록 요청하는 기법으로, 여러 경로를 통해 도출된 결과들의 일관성을 확인함으로써 답변의 신뢰성을 높입니다.

* 이 방법은 특히 수학 문제나 논리적 추론이 필요한 과제에서 효과적이며, 서로 다른 방법으로 도출된 결과가 일치하는지 검증함으로써 오류 가능성을 최소화할 수 있습니다.

* Self-Consistency의 장점은 답변의 정확성을 높일 수 있다는 것이지만, 여러 번의 계산과 추론이 필요하므로 처리 시간이 길어지고 컴퓨팅 리소스 사용량이 증가한다는 단점이 있습니다.

* 또한 이 기법은 Chain of Thought (CoT) 프롬프팅과 결합하여 사용할 경우 더욱 강력한 효과를 발휘할 수 있습니다.

- 논문: https://arxiv.org/abs/2203.11171


In [30]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
self_consistency_template = """
다음 문제를 세 가지 다른 방법으로 해결하시오:

문제: {question}

세 가지 풀이 방법:
1) 직접 계산 방법:
   - 주어진 숫자를 직접 계산

2) 비율 활용 방법:
   - 전체에 대한 비율로 계산

3) 단계별 분해 방법:
   - 문제를 작은 부분으로 나누어 계산

각 방법의 답안을 제시하고, 결과가 일치하는지 확인하시오.

답안:
"""

self_consistency_prompt = PromptTemplate(
   input_variables=["question"],
   template=self_consistency_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
self_consistency_chain = self_consistency_prompt | llm 
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

좋습니다! 주어진 문제를 세 가지 방법으로 해결해보겠습니다.

---

### 1) 직접 계산 방법

**단계 1:** 전체 학생 수 = 500명

**단계 2:** 5학년 학생 수 = 30% of 500 = 0.30 × 500 = **150명**

**단계 3:** 6학년 학생 수 = 20% of 500 = 0.20 × 500 = **100명**

**단계 4:** 5학년 중 수학 동아리 학생 = 60% of 150 = 0.60 × 150 = **90명**

**단계 5:** 5학년 중 과학 동아리 학생 = 나머지 = 150 - 90 = **60명**

**단계 6:** 6학년 중 수학 동아리 학생 = 70% of 100 = 0.70 × 100 = **70명**

**단계 7:** 6학년 중 과학 동아리 학생 = 나머지 = 100 - 70 = **30명**

**단계 8:** 과학 동아리 학생 총합 = 5학년 과학 + 6학년 과학 = 60 + 30 = **90명**

---

### 2) 비율 활용 방법

**전체 학생 수:** 500명

**5학년 학생 수:** 30% → 0.30 × 500 = 150명

**6학년 학생 수:** 20% → 0.20 × 500 = 100명

**5학년 과학 동아리 학생:** 40% (100% - 60%) of 150 = 0.40 × 150 = 60명

**6학년 과학 동아리 학생:** 30% (100% - 70%) of 100 = 0.30 × 100 = 30명

**과학 동아리 전체 학생 수:** 60 + 30 = **90명**

---

### 3) 단계별 분해 방법

- **단계 1:** 전체 학생 수 = 500명
- **단계 2:** 5학년 학생 = 150명, 6학년 학생 = 100명
- **단계 3:** 5학년 중 수학 동아리 = 60% → 0.60 × 150 = 90명
- **단계 4:** 5학년 과학 동아리 = 150 - 90 = 60명
- **단계 5:** 6학년 중 수학 동아리 = 70% → 0.70 × 10

# **Program-Aided Language (PAL)**

* PAL은 자연어 문제를 프로그래밍적 사고방식으로 접근하도록 하는 기법으로, 복잡한 문제를 코드나 의사코드 형태로 분해하여 해결하는 방식입니다. 이를 통해 문제 해결 과정을 더욱 구조화하고 체계적으로 만들 수 있습니다.

* 이 접근 방식의 큰 장점은 프로그래밍 언어의 정확성과 논리성을 활용하여 모호함을 줄이고, 각 단계를 명확하게 정의할 수 있다는 것입니다. 특히 수학적 계산, 데이터 처리, 알고리즘적 문제 해결에서 뛰어난 성능을 보입니다.

* PAL의 특징적인 점은 실제 실행 가능한 코드를 생성할 수 있다는 것으로, 이는 결과의 검증이 용이하고 필요한 경우 수정이나 최적화가 가능하다는 장점이 있습니다. 

- 논문: https://arxiv.org/pdf/2211.10435


In [33]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
pal_template = """
다음 문제를 Python 프로그래밍 방식으로 해결하시오:

문제: {question}

# 문제 해결을 위한 Python 스타일 의사코드:
def solve_problem():
    # 1. 변수 정의
    # - 주어진 값들을 변수로 저장
    
    # 2. 계산 과정
    # - 필요한 계산을 단계별로 수행
    
    # 3. 결과 반환
    # - 최종 결과 계산 및 반환
    
답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
pal_chain = pal_prompt | llm 
answer = pal_chain.invoke({"question": question})

print(answer.content)

```python
def solve_problem():
    # 1. 변수 정의
    total_students = 500
    percent_5th = 0.30
    percent_6th = 0.20

    # 2. 계산 과정
    # 5학년 학생 수
    num_5th = total_students * percent_5th
    # 6학년 학생 수
    num_6th = total_students * percent_6th

    # 5학년 학생들 중 수학 동아리 학생 수
    percent_math_5th = 0.60
    num_math_5th = num_5th * percent_math_5th
    # 5학년 학생들 중 과학 동아리 학생 수
    num_science_5th = num_5th - num_math_5th

    # 6학년 학생들 중 수학 동아리 학생 수
    percent_math_6th = 0.70
    num_math_6th = num_6th * percent_math_6th
    # 6학년 학생들 중 과학 동아리 학생 수
    num_science_6th = num_6th - num_math_6th

    # 과학 동아리 전체 학생 수
    total_science_club = num_science_5th + num_science_6th

    # 3. 결과 반환
    return int(total_science_club)

# 실행
print(solve_problem())
```


# **Reflexion**

* Reflexion은 AI가 자신의 이전 답변을 스스로 검토하고 평가하여 개선하는 메타인지적 프롬프팅 기법으로, 이를 통해 응답의 질을 점진적으로 향상시킬 수 있습니다.

* 이 방법은 AI가 자신의 답변에서 부족한 점, 오류, 또는 개선이 필요한 부분을 스스로 찾아내고 수정하도록 함으로써, 더 정확하고 완성도 높은 답변을 도출할 수 있게 합니다. 특히 복잡한 분석이나 창의적인 작업에서 효과적입니다.

* Reflexion의 강점은 AI가 자기 평가를 통해 지속적으로 개선된 결과물을 제공할 수 있다는 것이지만, 여러 번의 반복적인 검토와 수정 과정이 필요하므로 시간과 컴퓨팅 자원이 더 많이 소요될 수 있다는 제한점이 있습니다.

* 이 기법은 특히 글쓰기, 코드 리뷰, 분석 리포트 작성 등 높은 품질의 출력이 요구되는 작업에서 매우 유용하게 활용될 수 있습니다.

- 논문: https://arxiv.org/abs/2303.11366

In [34]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
reflexion_template = """
다음 문제에 대해 단계적으로 해결하여 초기 답안을 작성하고, 자체 평가 후 개선하시오:

문제: {question}

1단계: 초기 답안
---
[여기에 첫 번째 답안 작성]

2단계: 자체 평가
---
- 정확성 검토
- 논리적 오류 확인
- 설명의 명확성 평가
- 개선이 필요한 부분 식별

3단계: 개선된 답안
---
[평가를 바탕으로 개선된 답안 작성]

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
reflexion_chain = reflexion_prompt | ChatOpenAI(model='gpt-4.1-mini') 
answer = reflexion_chain.invoke({"question": question})

print(answer.content)

1단계: 초기 답안
---
주어진 조건에 따라 계산을 단계별로 진행해 보겠습니다.

1. 총 학생 수 = 500명

2. 5학년 학생 수 = 500명의 30%  
= 500 × 0.30 = 150명

3. 6학년 학생 수 = 500명의 20%  
= 500 × 0.20 = 100명

4. 5학년 학생 중 수학 동아리 학생 수 = 150명의 60%  
= 150 × 0.60 = 90명  
따라서, 5학년 과학 동아리 학생 수 = 150 - 90 = 60명

5. 6학년 학생 중 수학 동아리 학생 수 = 100명의 70%  
= 100 × 0.70 = 70명  
따라서, 6학년 과학 동아리 학생 수 = 100 - 70 = 30명

6. 과학 동아리 학생 수 전체 = 5학년 과학 동아리 + 6학년 과학 동아리  
= 60 + 30 = 90명

답: 과학 동아리에는 90명의 학생이 있습니다.

---

2단계: 자체 평가
---
- 정확성 검토: 문제에서 요구한 계산이 정확하게 이루어졌으며, 각 단계의 수치 계산도 올바르다.
- 논리적 오류 확인: 조건에 기반한 분류와 합산 과정에서 오류가 없다.
- 설명의 명확성 평가: 각 단계를 명확히 구분하여 설명하고 있으며, 이해하기 쉬운 구조이다.
- 개선이 필요한 부분 식별:  
  - 5학년과 6학년 외 다른 학년 학생 수가 언급되어 있지 않으나, 문제에서는 해당 학년들만 동아리 분포가 주어졌다.  
  - 다른 학년 학생들도 과학 동아리에 있을 가능성을 배제했는데, 문제 문맥상 5학년과 6학년만 고려하는 것으로 보인다.  
  - 따라서 문제 조건을 명확하게 해석해 5학년과 6학년에 해당하는 학생만 동아리에 참여한다는 점을 명시하면 더 좋겠다.

---

3단계: 개선된 답안
---
문제에서 주어진 조건에 따라 계산을 해 보겠습니다.

1. 학교 전체 학생 수는 500명입니다.

2. 이 중 5학년 학생 수는 전체의 30%이므로,  
   500 × 0.30 = 150명입니다.

3. 6학년 학생 수는 전체의 20%이므

### **[실습 2]** [실습 1]의 논리적 추론 문제를 gemma2:2b 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [39]:
question = """
농부가 늑대, 양, 양배추를 데리고 강을 건너야 함

제약조건:
1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
3. 보트에는 농부와 한 물건만 실을 수 있음

모두 안전하게 건너는데 몇 번 이동이 필요할까?
"""

In [51]:
# Self-Consisntency

### 프롬프트 템플릿
self_consistency_template = """
이 문제를 해결하기 위해 서로 다른 3가지의 생각의 흐름을 독립적으로 생성해 주세요. 
각 생각의 흐름은 처음부터 끝까지의 완전한 단계별 해결책이어야 합니다.
각 단계를 설명할 때는 [출발지에 남은 것] -> [도착지에 있는 것] 형태로 상태를 명시해주세요.
문제: {question}

세 가지 풀이방식 : 
1) 직접 계산 방식 :
- 주어진 숫자를 직접 계산 

2) 한 단계씩 나누어 생각하는 방식:
- 한 번 움직일 때마다 상태를 업데이트하며 생각

3) 역으로 생각하는 방식:
- 마지막 상태에서부터 역으로 생각하며 문제 해결

답안 : 
"""

self_consistency_prompt = PromptTemplate(
   input_variables=["question"],
   template=self_consistency_template
)

self_consistency_chain = self_consistency_prompt | gemma 
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

## 농부, 늑대, 양, 양배추 강 건너기 - 세 가지 풀이 방법

### 1. 직접 계산 방식 (불가능)

* 이 문제는 단순히 주어진 숫자를 계산하는 것이 아니라 복잡한 조건들을 만족하며 상태를 변화시켜야 하는 문제입니다. 따라서 직접 계산으로 해결할 수 없습니다.

### 2. 한 단계씩 나누어 생각하는 방식

1. **[농부, 양배추, 양, 늑대] -> [농부, 양배추]**: 농부는 양배추를 데리고 강을 건너갑니다.
2. **[농부, 양배추] -> [양배추, 늑대]**: 농부는 다시 돌아와 늑대를 데리고 간다.
3. **[양배추, 늑대] -> [양배추]**: 농부는 양배추를 데리고 강을 건너간다.
4. **[양배추] -> [양과 양배추]**: 농부는 양을 데리고 간다.
5. **[양과 양배추] -> [양, 늑대]**: 농부는 다시 돌아와 양을 데리고 간다.
6. **[양, 늑대] -> [늑대]**: 농부는 다시 돌아와 양을 데리고 간다.
7. **[늑대] -> [농부, 늑대]**: 농부는 다시 돌아와 양배추를 데리고 간다.

**총 이동: 7번**


### 3. 역으로 생각하는 방식

1. **[농부, 양, 양배추, 늑대]**: 모든 것이 안전하게 건너간 상태에서 시작합니다.
2. **[농부, 양, 양배추]**: 늑대는 이미 강 건너편에 있으므로, 농부는 양과 양배추를 데리고 다시 돌아옵니다.
3. **[농부, 양배추, 늑대]**: 농부는 늑대를 데리고 강을 건넌다.
4. **[양배추, 늑대]**: 농부는 양배추를 데리고 다시 돌아온다.
5. **[양배추]**: 농부는 양을 데리고 강을 건넌다.
6. **[양과 양배추]**: 농부는 양배추를 데리고 다시 돌아온다.
7. **[늑대, 양]**: 농부는 늑대를 데리고 강을 건넌다.

**총 이동: 7번**






In [44]:
# Program-Aided Language
pal_template = """
다음 문제를 Python 프로그래밍 방식으로 해결하시오:

문제: {question}

# 문제 해결을 위한 Python 스타일 의사코드:
def solve_problem():
    # 1. 변수 정의
    # - 주어진 값들을 변수로 저장
    
    # 2. 계산 과정
    # - 필요한 계산을 단계별로 수행
    
    # 3. 결과 반환
    # - 최종 결과 계산 및 반환
    
답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

pal_chain = pal_prompt | gemma3 
answer = pal_chain.invoke({"question": question})

print(answer.content)


```python
def solve_problem():
  """
  농부가 늑대, 양, 양배추를 데리고 강을 건너야 함
  """

  # 1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
  # 2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
  # 3. 보트에는 농부와 한 물건만 실을 수 있음

  # 문제 해결을 위한 Python 스타일 의사코드:
  # - 주어진 값들을 변수로 저장
  # - 계산 과정
  # - 결과 반환
  # - 최종 결과 계산 및 반환

  # 답안: 2번
```



In [45]:
# Reflexion
reflexion_template = """
다음 문제에 대해 단계적으로 해결하여 초기 답안을 작성하고, 자체 평가 후 개선하시오:

문제: {question}

1단계: 초기 답안
---
[여기에 첫 번째 답안 작성]

2단계: 자체 평가
---
- 정확성 검토
- 논리적 오류 확인
- 설명의 명확성 평가
- 개선이 필요한 부분 식별

3단계: 개선된 답안
---
[평가를 바탕으로 개선된 답안 작성]

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

reflexion_chain = reflexion_prompt | ChatOpenAI(model='gpt-4.1-mini') 
answer = reflexion_chain.invoke({"question": question})

print(answer.content)

1단계: 초기 답안
---
농부가 늑대, 양, 양배추를 강 건너편으로 안전하게 옮겨야 한다. 보트에는 농부와 한 물건만 탈 수 있고, 농부가 없으면 늑대와 양이 같이 있으면 늑대가 양을 잡아먹고, 양과 양배추가 같이 있으면 양이 양배추를 먹어버린다.

이 문제를 해결하기 위한 이동 방법(이동 횟수는 한 번 이동할 때마다 보트를 타고 건너거나 돌아오는 경우 모두 1번으로 계산한다):

1. 농부와 양을 강 건너편으로 이동 (1번째 이동)
2. 농부 혼자 귀환 (2번째 이동)
3. 농부와 늑대를 강 건너편으로 이동 (3번째 이동)
4. 농부와 양을 다시 강 건너편 출발지로 이동 (4번째 이동)
5. 농부와 양배추를 강 건너편으로 이동 (5번째 이동)
6. 농부 혼자 귀환 (6번째 이동)
7. 농부와 양을 강 건너편으로 재이동 (7번째 이동)

총 7번의 이동이 필요하다.

2단계: 자체 평가
---
- 정확성 검토: 제시한 이동 순서는 고전적인 해법과 일치하며 제약조건을 모두 만족한다.
- 논리적 오류 확인: 각 이동 후 양과 늑대, 양과 양배추가 함께 있으면서 위험한 상황이 발생하지 않는다.
- 설명의 명확성 평가: 각 단계가 어떠한 이유로 이루어지는지 간단한 이유 설명이 부족하다.
- 개선이 필요한 부분 식별: 각 이동이 왜 안전한지, 왜 특정 대상을 다시 데리고 가야 하는지에 대한 설명을 추가하면 이해가 더 쉬워질 것임.

3단계: 개선된 답안
---
농부가 늑대, 양, 양배추를 모두 안전하게 강을 건너려면 총 7번 이동이 필요하다.

이 문제는 농부가 없을 때 늑대가 양을 잡아먹고, 양이 양배추를 먹어버리는 제약 때문에 조심스럽게 옮겨야 한다. 다음은 안전한 이동 순서와 각 단계의 이유이다.

1. **농부와 양을 강 건너편으로 이동 (이동 1회)**  
   - 양을 먼저 데려감으로써 늑대와 양이 같이 남아 있지 않음.
2. **농부 혼자 돌아옴 (이동 2회)**  
   - 양만 강 건너편에 남고, 늑대와 양배추가 출발지에 함께 있음.  
   - 늑대